# LLM Investigation Notebook

This notebook sets up interactions with Gemini, ChatGPT, and Claude APIs.
It assumes a `.env` file is present in the same directory with valid API keys.

In [2]:
import os
from dotenv import load_dotenv
from google import genai
from openai import OpenAI
import anthropic
from typing import Optional, Literal, List
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

if not all([GEMINI_API_KEY, OPENAI_API_KEY, ANTHROPIC_API_KEY]):
    print("Warning: One or more API keys are missing in the .env file.")

## Configuration & Model Selection

In [3]:
# Model Configurations
# You can change these to other available models as needed.
GEMINI_MODEL = "gemini-1.5-pro-latest" # Or "gemini-1.5-flash"
GPT_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# Initialize Clients
if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)

if OPENAI_API_KEY:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)

if ANTHROPIC_API_KEY:
    anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Define Pydantic Model for Structured Output
class ClauseAnalysis(BaseModel):
    clause_text: str = Field(..., description="The exact clause text identified from the contract")
    is_identified: bool = Field(..., description="Whether the clause falls into a known Golden Clause category")
    category: Optional[str] = Field(None, description="The Golden Clause category if identified, e.g., 'Limitation of Liability'")
    risk_rating: Literal["Low", "Medium", "High"] = Field(..., description="The risk rating associated with the clause")
    justification: str = Field(..., description="Brief justification for the risk rating")

def call_llm(provider, prompt, system_instruction=None, response_model=None):
    """
    Unified wrapper to call different LLM providers with structured output support.
    provider: 'gemini', 'openai', or 'claude'
    prompt: The user prompt.
    system_instruction: Optional system instruction (supported differently by models).
    response_model: Optional Pydantic model for structured output.
    """
    try:
        if provider == "gemini":
            # Note: system_instruction usage depends on the library version, mostly supported in recent versions.
            generation_config = {}
            if response_model:
                generation_config["response_mime_type"] = "application/json"
                generation_config["response_schema"] = response_model

            model = genai.GenerativeModel(
                GEMINI_MODEL, 
                system_instruction=system_instruction,
                generation_config=generation_config
            )
            response = model.generate_content(prompt)
            if response_model:
                 # Gemini returns JSON text, we validate it against the model
                 return response_model.model_validate_json(response.text)
            return response.text

        elif provider == "openai":
            messages = []
            if system_instruction:
                messages.append({"role": "system", "content": system_instruction})
            messages.append({"role": "user", "content": prompt})
            
            if response_model:
                response = openai_client.beta.chat.completions.parse(
                    model=GPT_MODEL,
                    messages=messages,
                    response_format=response_model
                )
                return response.choices[0].message.parsed
            else:
                response = openai_client.chat.completions.create(
                    model=GPT_MODEL,
                    messages=messages
                )
                return response.choices[0].message.content

        elif provider == "claude":
            # Claude system prompts are top-level parameters
            messages = [{"role": "user", "content": prompt}]
            kwargs = {
                "model": CLAUDE_MODEL,
                "max_tokens": 1024,
                "messages": messages
            }
            if system_instruction:
                kwargs["system"] = system_instruction
                
            if response_model:
                # Construct tool definition from Pydantic schema
                schema = response_model.model_json_schema()
                tool_name = "print_analysis"
                tool_definition = {
                    "name": tool_name,
                    "description": "Output the analysis in structured format",
                    "input_schema": schema
                }
                kwargs["tools"] = [tool_definition]
                kwargs["tool_choice"] = {"type": "tool", "name": tool_name}
                
                response = anthropic_client.messages.create(**kwargs)
                
                # Extract tool use input
                for content in response.content:
                    if content.type == 'tool_use':
                        return response_model.model_validate(content.input)
                return "Error: No tool use found in Claude response"
            else:
                response = anthropic_client.messages.create(**kwargs)
                return response.content[0].text
        
        else:
            return "Error: Invalid provider specified."

    except Exception as e:
        return f"Error calling {provider}: {str(e)}"


## Individual Provider Examples

In [ ]:
# Gemini Direct Call
if GEMINI_API_KEY:
    print(f"Calling {GEMINI_MODEL}...")
    gemini_response = call_llm("gemini", "Explain the concept of 'force majeure' in one sentence.")
    print("Gemini Response:\n", gemini_response)
else:
    print("Gemini API Key not set.")

In [4]:
# ChatGPT Direct Call
if OPENAI_API_KEY:
    print(f"Calling {GPT_MODEL}...")
    openai_response = call_llm("openai", "Explain the concept of 'indemnification' in one sentence.")
    print("ChatGPT Response:\n", openai_response)
else:
    print("OpenAI API Key not set.")

Calling gpt-4o...
ChatGPT Response:
 Indemnification is a contractual obligation in which one party agrees to compensate another for any losses or damages incurred, effectively protecting them from financial liability.


In [ ]:
# Claude Direct Call
if ANTHROPIC_API_KEY:
    print(f"Calling {CLAUDE_MODEL}...")
    claude_response = call_llm("claude", "Explain the concept of 'termination for cause' in one sentence.")
    print("Claude Response:\n", claude_response)
else:
    print("Anthropic API Key not set.")

## Contract Review Example (Unified)
This section demonstrates how to use the selected models to review a dummy contract clause.

In [ ]:
contract_clause = """
The Service Provider shall not be liable for any indirect, special, or consequential damages arising out of this Agreement, except in cases of gross negligence or willful misconduct.
"""

system_instruction = "You are a legal assistant. Review the provided contract clause and identify potential risks for the Client."
prompt = f"Review the following clause:\n\n{contract_clause}"

# Choose which model to use for this specific task
SELECTED_PROVIDER = "gemini" # Change to "openai" or "claude" to test others

print(f"Reviewing with {SELECTED_PROVIDER}...")
review_result = call_llm(SELECTED_PROVIDER, prompt, system_instruction)
print("Review Result:\n", review_result)

In [5]:
system_prompt="""
You are an experienced commercial contracts lawyer and neutral risk analyst.

When provided with a contract or contract excerpt, your task is to:

1. Identify Golden Clauses
Determine whether the text relates to any of the following 10 Golden Clauses, based on legal substance rather than headings:
A. Payment Terms
B. Limitation of Liability
C. Indemnification
D. Governing Law & Jurisdiction
E. Data Privacy
F. Termination
G. Force Majeure
H. Intellectual Property
I. Confidentiality
J. Non-Solicitation

2. Assign Risk Rating
For each identified Golden Clause, assign a risk rating: Low, Medium, High

3. Risk Evaluation Principles
Assess risk by comparing the clause to an ideal, balanced commercial contract.

You MUST output the result in the valid JSON structure provided by the schema.
"""

In [6]:
contract_clause = """
The Service Provider shall not be liable for any indirect, special, or consequential damages arising out of this Agreement, except in cases of gross negligence or willful misconduct.
"""

prompt = f"Review the following clause:\n\n{contract_clause}"

# Choose which model to use for this specific task
SELECTED_PROVIDER = "openai" # "openai", "claude", "gemini"

print(f"Reviewing with {SELECTED_PROVIDER}...")
review_result = call_llm(SELECTED_PROVIDER, prompt, system_prompt, response_model=ClauseAnalysis)

if hasattr(review_result, 'model_dump_json'):
    print(f"Review Result ({SELECTED_PROVIDER} - JSON):\n", review_result.model_dump_json(indent=2))
else:
    print(f"Review Result ({SELECTED_PROVIDER} - Raw):\n", review_result)

Reviewing with openai...
Review Result (openai - JSON):
 {
  "clause_text": "The Service Provider shall not be liable for any indirect, special, or consequential damages arising out of this Agreement, except in cases of gross negligence or willful misconduct.",
  "is_identified": true,
  "category": "Limitation of Liability",
  "risk_rating": "Medium",
  "justification": "The clause limits the liability of the Service Provider for indirect, special, or consequential damages, which is common in commercial contracts to avoid excessive risks. However, it provides exceptions for cases of gross negligence or willful misconduct, which slightly increases the risk for the Service Provider. This balance of limiting broad liability except in severe cases is generally fair, although some risk remains due to the exclusions."
}
